 * Tiến hành crawl dữ liệu từ trang https://www.huffpost.com/

In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
import time

def is_leap_year(year):
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)    
  
def getDate(month, year):
    day_of_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if is_leap_year(year):
        day_of_month[2] = 29
    return day_of_month[month]

def getArticleFromPage(day, month, year):
    urlPage = 'https://www.huffpost.com/archive/{}-{}-{}'.format(str(year), str(month).zfill(2), str(day).zfill(2))
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    response = requests.get(urlPage, headers = headers)
    soupSite = BeautifulSoup(response.text, 'html.parser')
    soupArticle = soupSite.find_all("div", class_="card__headline")
    print(response)
    return soupArticle

def getHeadlineDataFormArticle(soupArticle):
    result = []
    for x in soupArticle:
        headline = x.find("div", class_="card__headline__text").get_text().replace("\n", "")
        article_link = x.find("a", class_="card__link yr-card-headline")['href']
        is_sarcastic = '0'
        result.append([headline, article_link, is_sarcastic])
    return result

def writeFile(headlineData):
    fileOutput = "huffpost.com.txt"
    f = open(fileOutput, "a", encoding="utf-8")
    for i in headlineData:
        strOut = i[0] + "|" + i[1] + "|" + i[2] + "\n"
        f.writelines(strOut)
    f.close()
        
def getDataFromPage(day, month, year):
    print("=>>>>> Date: {}/{}/{}:".format(day, month, year))
    soupArticle = getArticleFromPage(day, month, year)
    headlineData = getHeadlineDataFormArticle(soupArticle)
    writeFile(headlineData)
    print("Number of data: ",len(headlineData))
    print("\n")

if __name__ == "__main__":
    year = 2021
    year_stop = 2019
    while year > year_stop - 1:
        month = 12
        if year == 2021:
            month = 6
        while month:
            day = getDate(month, year)
            if year == 2021 and month == 6:
                day = 7
            while day: 
                getDataFromPage(day, month, year)
                day -= 1
            month -= 1
        year -= 1

=>>>>> Date: 7/6/2021:
<Response [200]>
Number of data:  47


=>>>>> Date: 6/6/2021:
<Response [200]>
Number of data:  16


=>>>>> Date: 5/6/2021:
<Response [200]>
Number of data:  31


=>>>>> Date: 4/6/2021:
<Response [200]>
Number of data:  62


=>>>>> Date: 3/6/2021:
<Response [200]>
Number of data:  57


=>>>>> Date: 2/6/2021:
<Response [200]>
Number of data:  66


=>>>>> Date: 1/6/2021:
<Response [200]>
Number of data:  62


=>>>>> Date: 31/5/2021:
<Response [200]>
Number of data:  34


=>>>>> Date: 30/5/2021:
<Response [200]>
Number of data:  25


=>>>>> Date: 29/5/2021:
<Response [200]>
Number of data:  24


=>>>>> Date: 28/5/2021:
<Response [200]>
Number of data:  52


=>>>>> Date: 27/5/2021:
<Response [200]>
Number of data:  66


=>>>>> Date: 26/5/2021:
<Response [200]>
Number of data:  75


=>>>>> Date: 25/5/2021:
<Response [200]>
Number of data:  59


=>>>>> Date: 24/5/2021:
<Response [200]>
Number of data:  42


=>>>>> Date: 23/5/2021:
<Response [200]>
Number of data:  23



* Chuyển dữ liệu từ file txt sang csv

In [ ]:
# Thêm thư viện cần thiết
import pandas as pd
# Đọc file csv bằng thư viện pandas
df = pd.read_csv("huffpost.com.txt", delimiter="|", names=['headline', 'article_link', 'is_sarcastic'])
df.to_csv('huffpost.csv')

* Tiến hành tổ chức dataset theo cùng format với dataset trên trang https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection

In [ ]:
def line(headline,article_link,is_sarcastic):
    line = '{"article_link": ' + '"' + article_link + '", "headline": "' + headline + '", "is_sarcastic": ' + "%i"%is_sarcastic + '}\n'
    return line

In [ ]:
import sys,os
f=open('huffpost.json','w')
for i in range(len(df)):
  if (df['headline'][i]!=None):
    f.write(line(df['headline'][i],df['article_link'][i],df['is_sarcastic'][i])+'\n')
f.close()